In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tftb.processing import WignerVilleDistribution

# Load your dataset
df = pd.read_csv('0hp_all_faults.csv')

# Use the 'DE_data' column for the WVD analysis
signal = df['DE_data'].values

# Define the time vector based on the length of your signal
sampling_rate = 48000  # 48 kHz for bearing end data
N = len(signal)
ts = np.arange(N) / sampling_rate

# Compute the Wigner-Ville Distribution
wvd = WignerVilleDistribution(signal, timestamps=ts)
tfr_wvd, t_wvd, f_wvd = wvd.run()

# Plot the Wigner-Ville Distribution
plt.figure()
plt.imshow(np.abs(tfr_wvd), aspect='auto', extent=[ts[0], ts[-1], 0, sampling_rate/2], origin='lower')
plt.title('Wigner-Ville Distribution')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.colorbar()
plt.show()


MemoryError: Unable to allocate 113. TiB for an array with shape (2782629, 2782629) and data type complex128